In [5]:
from datetime import datetime, timedelta
import pytz

def add_levothyroxine_schedule():
    calendar = GoogleCalendarManager()
    
    title = "💊 레보티록신 복용"
    description = "약물: 레보티록신\n복용 시간: 07:00\n⚠️ 정확한 시간에 복용하세요!"
    start_time = datetime(2025, 8, 3, 7, 0, 0, tzinfo=pytz.timezone("Asia/Seoul"))
    end_time = start_time + timedelta(minutes=30)

    event = {
        'summary': title,
        'description': description,
        'start': {
            'dateTime': start_time.isoformat(),
            'timeZone': 'Asia/Seoul'
        },
        'end': {
            'dateTime': end_time.isoformat(),
            'timeZone': 'Asia/Seoul'
        },
        'recurrence': [
            'RRULE:FREQ=DAILY;COUNT=30'
        ],
        'reminders': {
            'useDefault': False,
            'overrides': [
                {'method': 'popup', 'minutes': 10},
                {'method': 'popup', 'minutes': 0}
            ]
        }
    }

    try:
        created_event = calendar.service.events().insert(
            calendarId=calendar.CALENDAR_ID,
            body=event
        ).execute()
        print(f"✅ 일정이 생성되었습니다: {created_event.get('htmlLink')}")
    except Exception as e:
        print(f"❌ 일정 생성 중 오류 발생: {e}")

# 직접 실행
if __name__ == "__main__":
    add_levothyroxine_schedule()


✅ 일정이 생성되었습니다: https://www.google.com/calendar/event?eid=M25qcGt0YTB2bjloZ2JnOW5jOWJscXJ0MmtfMjAyNTA4MDJUMjIzMjAwWiBjX2NiYTNiODRiMjA3NGYwZGFkMmMxMTdkOTVmYTUzOTJiNmEyMzNjMTk5YTM4OTNhNjIxZWZhZjAwNDdmZDg2OWVAZw


In [10]:
import os
import json
import requests
import pytz
from datetime import datetime, timedelta
from dotenv import load_dotenv
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# .env 파일 로드
load_dotenv()

# ✅ IBM WatsonX 호출 함수
def call_watsonx_model(prompt_text: str) -> dict:
    API_KEY = os.getenv("IBM_API_KEY")
    DEPLOYMENT_URL = os.getenv("WATSONX_DEPLOYMENT_URL")
    IAM_TOKEN_URL = "https://iam.cloud.ibm.com/identity/token"

    token_response = requests.post(
        IAM_TOKEN_URL,
        headers={"Content-Type": "application/x-www-form-urlencoded"},
        data={
            "grant_type": "urn:ibm:params:oauth:grant-type:apikey",
            "apikey": API_KEY,
        },
        verify=False  # 개발 환경에서만 False 허용
    )
    if token_response.status_code != 200:
        raise Exception("❌ IAM 토큰 요청 실패:", token_response.text)

    iam_token = token_response.json()["access_token"]

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {iam_token}"
    }

    payload = {
        "parameters": {
            "prompt_variables": {
                "default": prompt_text
            }
        }
    }

    response = requests.post(
        DEPLOYMENT_URL,
        headers=headers,
        json=payload,
        verify=True
    )

    if response.status_code != 200:
        raise Exception("❌ WatsonX 모델 호출 실패:", response.text)

    return response.json()

# ✅ Google Calendar 연동 클래스
class GoogleCalendarManager:
    def __init__(self):
        self.SCOPES = ['https://www.googleapis.com/auth/calendar']
        self.CALENDAR_ID = os.getenv('CALENDAR_ID')
        self.CREDENTIALS_FILE = os.getenv('CREDENTIALS_FILE')
        self.TOKEN_FILE = os.getenv('TOKEN_FILE')
        self.KOREA_TZ = pytz.timezone('Asia/Seoul')
        self.service = self.authenticate()

    def authenticate(self):
        creds = None
        if os.path.exists(self.TOKEN_FILE):
            try:
                creds = Credentials.from_authorized_user_file(self.TOKEN_FILE, self.SCOPES)
            except:
                os.remove(self.TOKEN_FILE)
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(
                    self.CREDENTIALS_FILE, self.SCOPES)
                creds = flow.run_local_server(port=0)
            with open(self.TOKEN_FILE, 'w') as token:
                token.write(creds.to_json())
        return build('calendar', 'v3', credentials=creds)

    def create_event(self, title, start_time, end_time, description=''):
        if start_time.tzinfo is None:
            start_time = self.KOREA_TZ.localize(start_time)
        if end_time.tzinfo is None:
            end_time = self.KOREA_TZ.localize(end_time)

        event = {
            'summary': title,
            'description': description,
            'start': {
                'dateTime': start_time.isoformat(),
                'timeZone': 'Asia/Seoul',
            },
            'end': {
                'dateTime': end_time.isoformat(),
                'timeZone': 'Asia/Seoul',
            },
            'reminders': {
                'useDefault': False,
                'overrides': [
                    {'method': 'popup', 'minutes': 10},
                    {'method': 'popup', 'minutes': 0}
                ]
            }
        }

        try:
            created_event = self.service.events().insert(
                calendarId=self.CALENDAR_ID,
                body=event
            ).execute()
            print(f'✅ 일정이 생성되었습니다: {created_event.get("htmlLink")}')
        except HttpError as error:
            print(f'❌ 일정 생성 중 오류 발생: {error}')

# ✅ WatsonX 결과 기반 일정 생성
def extract_schedule_and_create_event(prompt_text: str):
    result = call_watsonx_model(prompt_text)

    # Step 1: generated_text 추출
    try:
        generated_text = result["results"][0]["generated_text"]
        print("🔍 WatsonX 원본 응답:\n", generated_text)
    except Exception as e:
        print("❌ WatsonX 응답에서 generated_text를 찾을 수 없습니다:", e)
        return

    # Step 2: JSON 부분 파싱
    try:
        json_start = generated_text.find('{')
        json_end = generated_text.rfind('}') + 1
        json_str = generated_text[json_start:json_end]

        # "일정끝" 같은 마무리 텍스트 제거 (있다면)
        if "일정끝" in json_str:
            json_str = json_str.split("일정끝")[0].strip()

        parsed = json.loads(json_str)
    except Exception as e:
        print("❌ JSON 파싱 실패:", e)
        print("🔎 추출된 JSON 문자열:\n", json_str)
        return

    # Step 3: calendar_events 처리
    events = parsed.get("calendar_events", [])
    if not events:
        print("❌ calendar_events가 없습니다.")
        return

    calendar = GoogleCalendarManager()
    for event in events:
        try:
            title = event.get("summary", "복약 일정")
            description = event.get("description", "")
            start_time = datetime.fromisoformat(event["start"]["dateTime"])
            end_time = datetime.fromisoformat(event["end"]["dateTime"])
            calendar.create_event(title, start_time, end_time, description=description)
        except Exception as e:
            print(f"❌ 일정 생성 중 오류: {e}")

# ✅ 메인 실행 함수
def main():
    print("💬 의사 처방 문장을 입력하세요:")
    user_input = input("예: 메트포르민 500mg 하루 2회 8시 20시, 2주간 복용\n> ")
    extract_schedule_and_create_event(user_input)

if __name__ == "__main__":
    main()


💬 의사 처방 문장을 입력하세요:


c:\Users\myeongbo\anaconda3\envs\ibm\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'iam.cloud.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


🔍 WatsonX 원본 응답:
                      {
  "medications": [
    {
      "name": "로라타딘",
      "frequency": 1,
      "times": ["08:00"],
      "original_text": "로라타딘 10mg 알레르기 증상 시 하루 1회 복용하세요. 최대 7일간"
    }
  ],
  "calendar_events": [
    {
      "summary": "💊 로라타딘 복용",
      "description": "약물: 로라타딘\n복용 시간: 08:00\n⚠️ 정확한 시간에 복용하세요!",
      "start": {
        "dateTime": "2025-08-03T08:00:00+09:00",
        "timeZone": "Asia/Seoul"
      },
      "end": {
        "dateTime": "2025-08-03T08:30:00+09:00",
        "timeZone": "Asia/Seoul"
      },
      "recurrence": [
        "RRULE:FREQ=DAILY;COUNT=7"
      ],
      "reminders": {
        "useDefault": false,
        "overrides": [
          {"method": "popup", "minutes": 10},
          {"method": "popup", "minutes": 0}
        ]
      }
    }
  ],
  "schedule_info": {
    "start_date": "2025-08-03",
    "duration_days": 7,
    "total_events": 1
  }
}
일정끝
✅ 일정이 생성되었습니다: https://www.google.com/calendar/event?eid=NTV1dHVpY25rYzc0djBvZ2c3b